<a href="https://colab.research.google.com/github/Jonathan-C-Barrett/GEOG5990M/blob/main/GEOG5003M_Final_Project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5003M Final Assignment (Template)

Student ID number:

GitHub repo link:

Word count limit= 1,500 words max (markdown cells only, excluding readme)

In [ ]:
# read in required packages

import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

import matplotlib.pyplot as plt

!pip install mapclassify

In [ ]:
#Read in Health Services Data
ahah = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/ahah_v4.csv')
ahah.head()

In [ ]:
#Select only columns relating to Health Services
health = ahah[['LSOA21CD', 'ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h']]
health.head()

In [69]:
# Data downloaded from https://geoportal.statistics.gov.uk/datasets/f23c50d2bffa4a8d9693bcf4ed72bdaf_0/explore?location=0.278866%2C-2.487028%2C0.00
shp =gpd.read_file('/content/Output_Areas_2021_EW_BFE_V9_-6709771492190738846.geojson')


In [ ]:
shp.info()

In [71]:
#Select only LSOAs in London Boroughs
london_shp= shp.loc[shp['LSOA21NM'].str.contains('City of London') |
                    shp['LSOA21NM'].str.contains('Barking and Dagenham')|
                    shp['LSOA21NM'].str.contains('Barnet')|
                    shp['LSOA21NM'].str.contains('Bexley')|
                    shp['LSOA21NM'].str.contains('Brent') |
                    shp['LSOA21NM'].str.contains('Bromley')|
                    shp['LSOA21NM'].str.contains('Camden')|
                    shp['LSOA21NM'].str.contains('Croydon')|
                    shp['LSOA21NM'].str.contains('Ealing')|
                    shp['LSOA21NM'].str.contains('Enfield')|
                    shp['LSOA21NM'].str.contains('Greenwich')|
                    shp['LSOA21NM'].str.contains('Hackney')|
                    shp['LSOA21NM'].str.contains('Hammersmith and Fulham')|
                    shp['LSOA21NM'].str.contains('Haringey')|
                    shp['LSOA21NM'].str.contains('Harrow')|
                    shp['LSOA21NM'].str.contains('Havering')|
                    shp['LSOA21NM'].str.contains('Hillingdon')|
                    shp['LSOA21NM'].str.contains('Hounslow')|
                    shp['LSOA21NM'].str.contains('Islington')|
                    shp['LSOA21NM'].str.contains('Kensington and Chelsea')|
                    shp['LSOA21NM'].str.contains('Kingston upon Thames')|
                    shp['LSOA21NM'].str.contains('Lambeth')|
                    shp['LSOA21NM'].str.contains('Lewisham')|
                    shp['LSOA21NM'].str.contains('Merton')|
                    shp['LSOA21NM'].str.contains('Newham')|
                    shp['LSOA21NM'].str.contains('Redbridge')|
                    shp['LSOA21NM'].str.contains('Richmond upon Thames')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Sutton')|
                    shp['LSOA21NM'].str.contains('Southwark')|
                    shp['LSOA21NM'].str.contains('Tower Hamlets')|
                    shp['LSOA21NM'].str.contains('Waltham Forest')|
                    shp['LSOA21NM'].str.contains('Wandsworth')|
                    shp['LSOA21NM'].str.contains('Westminster'),:]

In [72]:
london_shp.to_file('London.geojson')

In [ ]:
#Check London Borough worked
london_shp.info()

In [ ]:
#explore data on a map
london_shp.explore()

This is an example block of mardown text I want to reference <a href="#ref1">[1]</a>. I might need to add some more citations <a href="#ref2">[2]</a><a href="#ref2">[3]</a>



Data Visualisation
- Don't forget to present your final two data visualisation (one spatial and one non-spatial) and the justifications about the decisions you made whilst preparing and visualising the data.

## References
<p><a href="add_url_here">[1]</a>Footnote citation goes here</p>
<p><a href="https://github.com/FrancescaPontin/GEOG5990">[2]</a>GEOG5990M course materials</p>
<p><a href="add_url_here">[3]</a>Citation 3</p>